In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import gradio as gr
from PIL import Image
import os

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\krish\OneDrive\Desktop\dataset\TrashType_Image_Dataset", 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    r"C:\Users\krish\OneDrive\Desktop\dataset\TrashType_Image_Dataset", 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

class_names = list(train_generator.class_indices.keys())


Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [3]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, validation_data=val_generator, epochs=5)
model.save("garbage_model.h5")


C:\Users\krish\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\krish\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 60s 917ms/step - accuracy: 0.2562 - loss: 3.8804 - val_accuracy: 0.4473 - val_loss: 1.3927
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 59s 921ms/step - accuracy: 0.5142 - loss: 1.2626 - val_accuracy: 0.4712 - val_loss: 1.3823
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 57s 882ms/step - accuracy: 0.6453 - loss: 1.0011 - val_accuracy: 0.5209 - val_loss: 1.3627
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 53s 828ms/step - accuracy: 0.7845 - loss: 0.6534 - val_accuracy: 0.5626 - val_loss: 1.3818
Epoch 5/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 51s 802ms/step - accuracy: 0.8894 - loss: 0.3708 - val_accuracy: 0.5765 - val_loss: 1.5214


In [4]:
model = tf.keras.models.load_model("garbage_model.h5")

def classify_image(img):
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = img_array.reshape(1, 224, 224, 3)
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    return class_names[class_index]


In [ ]:
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Garbage Classification",
    description="Upload a garbage image to classify it as Organic, Recyclable, or Hazardous"
)

iface.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
